# mp4 파일 전송

In [2]:
import socket

sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
ip = '203.255.56.63'
port = 44444
sock.bind((ip, port))
sock.listen(1)
client, addr = sock.accept()

My_video = '/Users/winlab/Desktop/Summer.mp4'
video = open(My_video,'rb')
data = video.read()
header = data.find(b'mdat') + 5

def frag(l):
    n = 1
    while 1:
        if l == 0:
            return 0
        if l/n >= header:
            n += 1
        else:
            return n

n_d = frag(len(data))

client.send(str(header).encode())
client.send(str(n_d).encode())

for i in range(n_d):
    client.send(data[i*header:(i+1)*header])

sock.close()

# 인덱싱된 mp4 파일 전송

In [1]:
import numpy as np
import socket

def indexing_mp4(v_path1, v_path2):
    with open(v_path1, 'rb') as v1:  # 첫번째 영상 바이너리 형식으로 읽어오기
        v1_data = v1.read()
        
    with open(v_path2, 'rb') as v2: # 두번째 영상 바이너리 형식으로 읽어오기
        v2_data = v2.read()
    
    data_idx1 = v1_data.find(b'mdat')
    header_idx1 = v1_data.find(b'moov')
    
    if data_idx1 < header_idx1:
        print("can't stream")
        return
    
    data_idx2 = v2_data.find(b'mdat')
    header_idx2 = v2_data.find(b'moov')
    
    if data_idx2 < header_idx2:
        print("can't stream")
        return
    
    np_v1 = np.frombuffer(v1_data, dtype='uint8') # 바이너리 해석
    np_v2 = np.frombuffer(v2_data, dtype='uint8')

    l1 = len(np_v1)
    l2 = len(np_v2)
    if l1 > l2:
        xor_len = l2
        add_data = np_v1[l2:l1]
    else:
        xor_len = l1
        add_data = np_v2[l1:l2]
        
    # 데이터의 길이를 비교
    
    xor_v = np.bitwise_xor(np_v1[:xor_len], np_v2[:xor_len]) # xor 연산
    xor_v_data = np.append(xor_v, add_data)

    xor_v_bin = xor_v_data.tostring() # 택 데이터 바이너리화
    return xor_v_bin, data_idx1 + 5, data_idx2 + 5, l1, l2

v_path1 = '/Users/winlab/Desktop/Yes.mp4'
v_path2 = '/Users/winlab/Desktop/Summer.mp4'
data, header1, header2, l1, l2 = indexing_mp4(v_path1, v_path2)

def frag(l, header):
    n = 1
    while 1:
        if l == 0:
            return 0
        if l/n >= header:
            n += 1
        else:
            return n
        
sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
ip = '203.255.56.63'
port = 44444
sock.bind((ip, port))
sock.listen(1)
client, addr = sock.accept()

file_name = client.recv(1024).decode()

if file_name == 'Summer':  # 클라이언트가 가지고 있는 동영상 이름
    header = header1
    l = l1
    n_d = frag(l, header)
elif file_name == 'Yes':
    header = header2
    l = l2
    n_d = frag(l, header)

client.send(str(header).encode())
client.send(str(n_d).encode())

for i in range(n_d):
    client.send(data[i*header:(i+1)*header])

client.close()

# 인덱싱된 mp4 파일 여러명한테 전송

In [1]:
import numpy as np
import socket
import threading

def indexing_mp4(v_path1, v_path2):
    with open(v_path1, 'rb') as v1:  # 첫번째 영상 바이너리 형식으로 읽어오기
        v1_data = v1.read()
        
    with open(v_path2, 'rb') as v2: # 두번째 영상 바이너리 형식으로 읽어오기
        v2_data = v2.read()
    
    data_idx1 = v1_data.find(b'mdat')
    header_idx1 = v1_data.find(b'moov')
    
    if data_idx1 < header_idx1:
        print("can't stream")
        return
    
    data_idx2 = v2_data.find(b'mdat')
    header_idx2 = v2_data.find(b'moov')
    
    if data_idx2 < header_idx2:
        print("can't stream")
        return
    
    np_v1 = np.frombuffer(v1_data, dtype='uint8') # 바이너리 해석
    np_v2 = np.frombuffer(v2_data, dtype='uint8')

    l1 = len(np_v1)
    l2 = len(np_v2)
    if l1 > l2:
        xor_len = l2
        add_data = np_v1[l2:l1]
    else:
        xor_len = l1
        add_data = np_v2[l1:l2]
        
    # 데이터의 길이를 비교
    
    xor_v = np.bitwise_xor(np_v1[:xor_len], np_v2[:xor_len]) # xor 연산
    xor_v_data = np.append(xor_v, add_data)

    xor_v_bin = xor_v_data.tostring() # 택 데이터 바이너리화
    return xor_v_bin, data_idx1 + 5, data_idx2 + 5, l1, l2

v_path1 = '/Users/winlab/Desktop/Yes.mp4'
v_path2 = '/Users/winlab/Desktop/Summer.mp4'
data, header1, header2, l1, l2 = indexing_mp4(v_path1, v_path2)

def frag(l, header):
    n = 1
    while 1:
        if l == 0:
            return 0
        if l/n >= header:
            n += 1
        else:
            return n
        
sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
ip = '203.255.56.63'
port = 44444
sock.bind((ip, port))
sock.listen(1)

def thread():
    client, addr = sock.accept()
    
    t = threading.Thread(target=thread)
    t.start()
    
    file_name = client.recv(1024).decode()

    if file_name == 'Summer':  # 클라이언트가 가지고 있는 동영상 이름
        header = header1
        l = l1
        n_d = frag(l, header)
    elif file_name == 'Yes':
        header = header2
        l = l2
        n_d = frag(l, header)

    client.send(str(header).encode())
    client.send(str(n_d).encode())

    for i in range(n_d):
        client.send(data[i*header:(i+1)*header])

    client.close()
    
thread()